In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [2]:
%load_ext autoreload
%autoreload 2 

project_dir = os.path.dirname(os.getcwd())
print('project direcotry is ', project_dir)
if project_dir not in sys.path:
    sys.path.append(project_dir)
print(sys.path)

project direcotry is  /home/yuan/bio/antibody-antigen-complex
['/home/yuan/anaconda3/envs/openfold-env/lib/python310.zip', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/lib-dynload', '', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages/openfold-2.2.0-py3.10-linux-x86_64.egg', '/home/yuan/bio/antibody-antigen-complex']


## view structure

In [3]:
# get *pdb
from src.download import Download

pdb_id = '1CLY'
pdb_file = Download.download_pdb(pdb_id)

../pdb/1CLY.pdb is ready.


In [4]:
import py3Dmol
# Create a view
view = py3Dmol.view(width=400, height=300)
view.addModel(open(pdb_file).read(), 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## parse complex

In [5]:
# get chains
from src.parse_abag import ParseAbAg

outputs_dir = '../outputs'
p = ParseAbAg(pdb_file, outputs_dir)
# p.structure.header

# get chains
chains = p.get_chains()
pprint(chains)

Successfully retrieve structure of 1CLY
outputs dir:  ../outputs/1C/1CLY
[{'chains': [{'AA residues': 217,
              'chain_id': 'L',
              'seq': 'LMTQIPVSLPVSLGDQASISCRSSQIIVHNNGNTYLEWYLQKPGQSPQLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYYCFQGSHVPFTFGSGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRGEC'},
             {'AA residues': 220,
              'chain_id': 'H',
              'seq': 'EVNLVESGGGLVQPGGSLKVSCVTSGFTFSDYYMYWVRQTPEKRLEWVAYISQGGDITDYPDTVKGRFTISRDNAKNSLYLQMSRLKSEDTAMYYCARGLDDGAWFAYWGQGTLVTVSVASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPQPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKRVEP'},
             {'AA residues': 4, 'chain_id': 'A', 'seq': ''}],
  'model_id': 0}]


In [6]:
p.structure.header['compound']

{'1': {'misc': '',
  'molecule': 'igg fab (human igg1, kappa)',
  'chain': 'l',
  'fragment': 'chimeric fragment (br96)',
  'synonym': 'cbr96 fab (immunoglobulin)',
  'engineered': 'yes',
  'other_details': 'complexed with lewis y nonoate methyl ester'},
 '2': {'misc': '',
  'molecule': 'igg fab (human igg1, kappa)',
  'chain': 'h',
  'fragment': 'chimeric fragment (br96)',
  'synonym': 'cbr96 fab (immunoglobulin)',
  'engineered': 'yes',
  'other_details': 'complexed with lewis y nonoate methyl ester'}}

In [7]:
# filter: is antibody-antigen complex
p.filter_antibody_antigen()
p.update_chains('chain_type')
print(p.structure_id)
pprint(p.chains[0]['chains'][0])

Pairwise chains of 1CLY are updated.
1CLY
{'AA residues': 217,
 'chain_id': 'L',
 'chain_type': 'single',
 'seq': 'LMTQIPVSLPVSLGDQASISCRSSQIIVHNNGNTYLEWYLQKPGQSPQLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYYCFQGSHVPFTFGSGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRGEC'}


In [8]:
# split chains
p.chains_to_pdb()

Saved chain ['L'] to ../outputs/1C/1CLY/1CLY_L.pdb
Saved chain ['H'] to ../outputs/1C/1CLY/1CLY_H.pdb
Saved chain ['L', 'H'] to ../outputs/1C/1CLY/1CLY_L_H.pdb


In [9]:
p.bounded

[{'type': 'single-single',
  'chains': [['L'], ['H'], ['L', 'H']],
  'pdb_files': ['../outputs/1C/1CLY/1CLY_L.pdb',
   '../outputs/1C/1CLY/1CLY_H.pdb',
   '../outputs/1C/1CLY/1CLY_L_H.pdb']}]

In [10]:
p.save_annot()

https://www.ebi.ac.uk/pdbe/api/mappings/1cly
Save annotation of 1CLY: ../outputs/1C/1CLY/meta.json


## SASA

In [11]:
# calculate SASA
sasa_file = p.build_freesasa()

In [12]:
from src.analyze_sasa import AnalyzeSasa

p2= AnalyzeSasa(sasa_file)
p2.print_meta()
print(list(p2.data[0]['delta_total_sasa']))

key1 ['type', 'chains', 'pdb_files', 'sasa_files', 'delta_total_sasa']
single chain L ~ single chain H
------------------------------
['delta1', 'delta1_file', 'delta2', 'delta2_file']


In [13]:
# retrieve sequences
motifs = p2.retrieve_seq()
print('output file: ', motifs[1])
motifs

output file:  ../outputs/1C/1CLY/seq_sasa_delta-0_span-30.p


([('single',
   [{'seq': 'HNNGNTYLEWYLQKPGQSPQLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYYCFQGSHVPFTFGSGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFN',
     'sig_res': 53,
     'start': (28, '27D'),
     'end': (212, '210'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}],
   'single',
   [{'seq': 'YWVRQTPEKRLEWVAYISQGGDITDYPDTVKGRFTISRDNAKNSLYLQMSRLKSEDTAMYYCARGLDDGAWFAYWGQGTLVTVSVASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPQPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDK',
     'sig_res': 58,
     'start': (34, '35'),
     'end': (214, '221'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}])],
 '../outputs/1C/1CLY/seq_sasa_delta-0_span-30.p')

In [14]:
# dataset for ML
seq_sasa_file = motifs[1]
AnalyzeSasa(seq_sasa_file).build_dataset()

[{'input': 'YWVRQTPEKRLEWVAYISQGGDITDYPDTVKGRFTISRDNAKNSLYLQMSRLKSEDTAMYYCARGLDDGAWFAYWGQGTLVTVSVASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPQPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDK',
  'input_label': 'single',
  'output': 'HNNGNTYLEWYLQKPGQSPQLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYYCFQGSHVPFTFGSGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFN',
  'output_label': 'single'}]

In [15]:
# complex
p2.print_delta_sasa(0)

complex: single chain L ~ single chain H
------------------------------
single chain:
    chain_id res_no  res aa       value
28         L    27D  HIS  H    2.509655
34         L     32  TYR  Y   39.991922
36         L     34  GLU  E   31.692940
38         L     36  TYR  Y   39.379990
40         L     38  GLN  Q   48.651583
43         L     41  GLY  G    3.484001
44         L     42  GLN  Q    3.137182
45         L     43  SER  S   56.356151
46         L     44  PRO  P   71.418806
47         L     45  GLN  Q    5.407762
48         L     46  LEU  L   34.578631
51         L     49  TYR  Y   56.241885
52         L     50  LYS  K   32.690257
55         L     53  ASN  N    1.998656
57         L     55  PHE  F   61.486548
87         L     85  VAL  V    2.703881
89         L     87  TYR  Y   51.815745
91         L     89  PHE  F   30.115860
93         L     91  GLY  G   13.754998
96         L     94  VAL  V   45.965974
97         L     95  PRO  P   25.686868
98         L     96  PHE  F  105.5

## distance

In [16]:
# calculate distance
dist_file = p.build_dist()

In [17]:
from src.analyze_dist import AnalyzeDist

max_span = 30
p3= AnalyzeDist(dist_file, max_span=max_span)
p3.print_meta()
print(list(p3.data[0]['dist']))

key1 ['type', 'chains', 'pdb_files', 'dist']
single chain L ~ single chain H
------------------------------
['dist1', 'dist1_file', 'dist2', 'dist2_file']


In [18]:
# retrieve sequences
motifs = p3.retrieve_seq()
motifs

([('single',
   [{'seq': 'YLEWYLQKPGQSPQLLIYKVSNRF',
     'sig_res': 12,
     'start': (29, 32),
     'end': (52, 55),
     'dist_cutoff': 5,
     'max_span': 30},
    {'seq': 'YCFQGSHVPFTFGSGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFN',
     'sig_res': 39,
     'start': (84, 87),
     'end': (207, 210),
     'dist_cutoff': 5,
     'max_span': 30}],
   'single',
   [{'seq': 'YWVRQTPEKRLEWVAYIQGGDITDYP',
     'sig_res': 11,
     'start': (34, 35),
     'end': (59, 60),
     'dist_cutoff': 5,
     'max_span': 30},
    {'seq': 'YCARGLDDGAAYWGQGTLVTVSVASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPQPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDK',
     'sig_res': 41,
     'start': (90, 91),
     'end': (208, 221),
     'dist_cutoff': 5,
     'max_span': 30}])],
 '../outputs/1C/1CLY/seq_dist_dist-5_span-30.p')

In [19]:
seq_dist_file = motifs[1]
AnalyzeDist(seq_dist_file).build_dataset()

[{'input': 'YWVRQTPEKRLEWVAYIQGGDITDYP | YCARGLDDGAAYWGQGTLVTVSVASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPQPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDK',
  'input_label': 'single',
  'output': 'YLEWYLQKPGQSPQLLIYKVSNRF | YCFQGSHVPFTFGSGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFN',
  'output_label': 'single'}]

In [20]:
# complex
p3.print_min_dist(0)

------------------------------
     res_no  res atom      value aa
0         3  LEU    C   9.266197  L
1         4  MET    C   7.320885  M
2         5  THR    C   8.606447  T
3         6  GLN    C   8.563429  Q
4         7  ILE    C  10.884436  I
..      ...  ...  ...        ... ..
207     210  ASN    C   3.730306  N
208     211  ARG    C   8.118162  R
209     212  GLY    C   8.859418  G
210     213  GLU    C   9.953323  E
211     214  CYS    C   9.105592  C

[212 rows x 5 columns]
------------------------------
     res_no  res atom     value aa
34       35  TYR    C  4.734736  Y
36       37  VAL    C  4.122544  V
38       39  GLN    C  2.724256  Q
42       43  LYS    C  2.810833  K
43       44  ARG    C  3.997206  R
44       45  LEU    C  3.535442  L
45       46  GLU    C  4.117050  E
46       47  TRP    C  3.705781  W
49       50  TYR    C  3.907830  Y
57       58  ASP    C  3.843943  D
59       60  PRO    C  4.531843  P
90       91  TYR    C  3.543213  Y
95       96  LEU    C  3.64